In [10]:
# default dependencies
import pandas as pd
import duckdb

# services
from snomed_characterization.services.import_duckdb_concepts_to_snomed_graph import (
    ImportDuckDBConceptsToSNOMEDGraph,
)

from snomed_characterization.services.load_snomed_concepts_from_duckdb import (
    LoadSNOMEDConceptsFromDuckdb,
)

#graph
from snomed_characterization.snomed_graph import SNOMEDGraph


In [11]:
#get data

service = LoadSNOMEDConceptsFromDuckdb("data/data.duckdb")
raw_concepts = service.call()

In [13]:
raw_concepts, len(raw_concepts)

([RawSNOMEDConcept(concept_id=8507.0, concept_name='MALE', domain_id='Gender', vocabulary_id='Gender', concept_class_id='Gender', standard_concept='S', concept_code='M', valid_start_date=Timestamp('1970-01-01 00:00:00'), valid_end_date=Timestamp('2099-12-31 00:00:00'), invalid_reason=None),
  RawSNOMEDConcept(concept_id=8532.0, concept_name='FEMALE', domain_id='Gender', vocabulary_id='Gender', concept_class_id='Gender', standard_concept='S', concept_code='F', valid_start_date=Timestamp('1970-01-01 00:00:00'), valid_end_date=Timestamp('2099-12-31 00:00:00'), invalid_reason=None),
  RawSNOMEDConcept(concept_id=24660.0, concept_name='Acute tonsillitis', domain_id='Condition', vocabulary_id='SNOMED', concept_class_id='Clinical Finding', standard_concept='S', concept_code='17741008', valid_start_date=Timestamp('2072-02-02 00:00:00'), valid_end_date=Timestamp('2170-01-02 00:00:00'), invalid_reason=None),
  RawSNOMEDConcept(concept_id=25297.0, concept_name='Acute pharyngitis', domain_id='Cond

there are 102 concepts (?)

In [14]:
#import data 
db_path = "data/data.duckdb"
g = SNOMEDGraph()
importer_service = ImportDuckDBConceptsToSNOMEDGraph(db_path, g)

_ = importer_service.call()

# nx graph
graph = g.graph

print(len(graph.nodes))


1838


In [8]:
zero_out_degree_nodes = [node for node, degree in graph.out_degree() if degree == 0] 

In [9]:
zero_out_degree_nodes

[8532.0,
 8507.0,
 73553.0,
 81454.0,
 140168.0,
 201826.0,
 372887.0,
 376208.0,
 432661.0,
 80180.0,
 133835.0,
 135772.0,
 198572.0,
 321588.0,
 374034.0,
 375519.0,
 435783.0,
 4002898.0,
 4027255.0,
 4028062.0,
 4063381.0,
 4047269.0,
 4160899.0,
 4187067.0,
 4280071.0,
 440543.0,
 441542.0,
 441284.0,
 3655971.0,
 4013263.0,
 4032881.0,
 4088164.0,
 4110709.0,
 4132926.0,
 4172436.0,
 4182210.0,
 4189855.0,
 4190185.0,
 4229909.0,
 4274025.0,
 4309188.0,
 4324765.0,
 4338039.0,
 320420.0,
 316866.0,
 320136.0,
 435786.0,
 80809.0,
 255919.0,
 376125.0,
 435789.0,
 441840.0,
 441848.0,
 4005799.0,
 4006969.0,
 4112341.0,
 4149084.0,
 4181194.0,
 4184117.0,
 4224741.0,
 4213101.0,
 4280354.0,
 4287548.0,
 4328821.0,
 4329847.0,
 37206233.0,
 40481826.0,
 141253.0,
 201909.0,
 318736.0,
 375415.0,
 379019.0,
 436670.0,
 196724.0,
 199860.0,
 381273.0,
 381295.0,
 432586.0,
 442793.0,
 442983.0,
 443785.0,
 4008576.0,
 4024552.0,
 4031814.0,
 4116811.0,
 4139452.0,
 4147164.0,
 41622

In [ ]:
# exploring out of degree nodes


service = LoadSNOMEDConceptsFromDuckdb("data/data.duckdb")

data = service.call()
concepts_df = service._load_concepts()
for row in concepts_df.itertuples():
    print(row.concept_id)
